<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/micro_seq_bn_helpfulness_tapt_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TEST RUN
* CHANGE epochs to 100
* CHANGE adapter_name


# Summary
This TAPT pretrains a seq_bn adapter on the micro amazon helpfulness dataset.

# Settings
* LR of 0.0001
* No warmup
* Set batch size to 21, gradient_accumulation_steps=2, ~6 reviews per row = 252 batch size (paper has 256)
* Adapter - seq_bn
  * torch_compile=False (Because no adapter)
* early_stopping_patience=10
* 100 epoch




# Setup

In [1]:
from time import time
start_time = time()

In [2]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 36.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-no

In [4]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_MICRO_helpfulness_dataset_condensed"

dataset = load_dataset(dataset_name)

print(dataset_name)

Generating train split:   0%|          | 0/1974 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1989 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

BigTMiami/amazon_MICRO_helpfulness_dataset_condensed


In [5]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import SeqBnConfig

adapter_config = SeqBnConfig()
adapter_name = "micro_helpfulness_tapt_pretrain_seq_bn"
adapter_hub_name = f"TEST_{adapter_name}_adapter"
print(adapter_hub_name)

TEST_micro_helpfulness_tapt_pretrain_seq_bn_adapter


In [7]:
# Add a new adapter
model.add_adapter(adapter_name, config=adapter_config)

# Add head for masked language modeling
model.add_masked_lm_head(adapter_name)

# Set the adapter to be used for training
model.train_adapter(adapter_name)

print(f"Adapter Name: {adapter_name}")

Adapter Name: micro_helpfulness_tapt_pretrain_seq_bn


In [8]:
summary = model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
micro_helpfulness_tapt_pretrain_seq_bnbottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=adapter_hub_name,
    overwrite_output_dir=True,
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=1,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [13]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [14]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [15]:
train_results = trainer.train()
print(train_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,9.998400,7.583788


TrainOutput(global_step=47, training_loss=9.998377373877991, metrics={'train_runtime': 70.386, 'train_samples_per_second': 28.045, 'train_steps_per_second': 0.668, 'total_flos': 532587535699968.0, 'train_loss': 9.998377373877991, 'epoch': 1.0})


In [17]:
eval_results = trainer.evaluate(dataset["test"])
print(eval_results)

{'eval_loss': 7.618783950805664, 'eval_runtime': 22.3583, 'eval_samples_per_second': 88.96, 'eval_steps_per_second': 4.249, 'epoch': 1.0}


In [18]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name,
    overwrite_adapter_card=True,
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/TEST_micro_helpfulness_tapt_pretrain_seq_bn_adapter/commit/1223693aaa072e7464a283a2790b706770fab3a9', commit_message='Upload model', commit_description='', oid='1223693aaa072e7464a283a2790b706770fab3a9', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

Total Time: 5 minutes OR 303 seconds


In [20]:
# print("Disconnecting Session")
# from google.colab import runtime
# runtime.unassign()